In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import unicodedata
import string
import re
import random

import time
import datetime
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import spacy
import numpy as np

# from redditscore.tokenizer import CrazyTokenizer

In [0]:
# ! pip install transformers
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py

In [3]:
manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cuda


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
n_gpu = torch.cuda.device_count()

cuda


In [0]:
# import torch_xla
# import torch_xla.core.xla_model as xm

# device = xm.xla_device()
# print(device)

In [0]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download fr_core_news_sm

In [10]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

import fr_core_news_sm
import en_core_web_sm

spacy_fr = fr_core_news_sm.load()
spacy_en = en_core_web_sm.load()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.7MB 1.8MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=6050d4465f8a4a7bf60d5dce927247d354e47dfef1872cb6f0bd9b96fcbd6824
  Stored in directory: /tmp/pip-ephem-wheel-cache-bw4a5fvi/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [0]:
import torchtext

In [0]:
def tokenize_fr(text):
    """
    Tokenizes French text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_fr.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [0]:
import pandas as pd
import csv

In [0]:
with open("/content/drive/My Drive/Colab Notebooks/data/MTNT/valid/valid.fr-en.tsv") as data:
  with open('/content/drive/My Drive/Colab Notebooks/data/MTNT/train/output_val.tsv', 'wt') as out_file:
    rd = csv.reader(data, delimiter="\t", quotechar='"')
    for l in rd:
      if len(l) < 3:
        print(l)
        continue
      tsv_writer = csv.writer(out_file, delimiter='\t')
      tsv_writer.writerow([l[2], l[1]])

In [0]:
with open("/content/drive/My Drive/Colab Notebooks/MTNT/test/test.fr-en.tsv") as data:
  with open('/content/drive/My Drive/Colab Notebooks/MTNT/train/output_test.tsv', 'wt') as out_file:
    rd = csv.reader(data, delimiter="\t", quotechar='"')
    for l in rd:
      if len(l) < 3:
        print(l)
        continue
      tsv_writer = csv.writer(out_file, delimiter='\t')
      tsv_writer.writerow([l[2], l[1]])

In [0]:
with open("/content/drive/My Drive/Colab Notebooks/MTNT/train/train.fr-en.tsv") as data:
  with open('/content/drive/My Drive/Colab Notebooks/MTNT/train/output.tsv', 'wt') as out_file:
    rd = csv.reader(data, delimiter="\t", quotechar='"')
    for l in rd:
      if len(l) < 3:
        print(l)
        continue
      tsv_writer = csv.writer(out_file, delimiter='\t')
      tsv_writer.writerow([l[2], l[1]])

['4312', '*\t*']
['5167', '2008 !\t2008!']
['6545', ':D\t:D\n3823\tDe 2000 à 2012 tu veux dire.\tFrom 2000 to 2012 you mean.\n4141\tDear French Redditors, could you help me to find this song?\tDear French Redditors, could you help me to find this song?\n6637\t* Death roes of the Republic -> Sur la chute de la république romaine.\t* Death roes of the Republic -> About the fall of the Romanian republic.\n9529\t* Death roes of the Republic -> Sur la chute de la république romaine.\t* Death roes of the Republic -> On the fall of the Roman republic.\n9807\tDébat : « Balance ton transport », ou quand la RATP diabolise les animaux pour lutter contre le harcèlement sexuel  En quoi mettre un homme harceleur serait caricaturale ou problématique ?   Illustrer des comportements nocifs par des représentations réalistes est quelque chose de normal (voir sécurité routière par exemple).\tDebate: Throw our your transportation," or when the RATP demonizes animals to fight against sexual harassment. How 

In [13]:
SRC = torchtext.data.Field(tokenize = tokenize_fr, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TRG = torchtext.data.Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

train, val, test = torchtext.data.TabularDataset.splits(
    path='/content/drive/My Drive/Colab Notebooks/Trends Project/data/', train='output.tsv',validation='output_val.tsv', test='output_test.tsv', 
    format='tsv', skip_header=True, fields=[('TRG', TRG), ('SRC', SRC)])

print(f"Number of training examples: {len(train.examples)}")
print(f"Number of validation examples: {len(val.examples)}")
print(f"Number of testing examples: {len(test.examples)}")

TRG.build_vocab(train, max_size = 6000)
SRC.build_vocab(train, max_size = 6000)
# TRG.build_vocab(train, min_freq = 5)
# SRC.build_vocab(train, min_freq = 5)

print(f"Unique tokens in source (fr) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

train_iter, val_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train, val, test), # we pass in the datasets we want the iterator to draw data from
    batch_sizes=(4, 64, 64),device = device,
    sort_key=lambda x: len(x.SRC), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=True)

Number of training examples: 18941
Number of validation examples: 875
Number of testing examples: 1021
Unique tokens in source (fr) vocabulary: 6004
Unique tokens in target (en) vocabulary: 6004


In [14]:
print(vars(train.examples[-1])['TRG'])

['perpignan', 'woman', 'lives', 'for', '12', 'years', 'without', 'running', 'water', '.', 'saur', 'sentences', '.', 'these', '€', '200', 'should', 'be', 'more', 'or', 'less', 'the', 'annual', 'consumption', '(', 'for', '100m3', ')', '.', 'it', "'s", 'water', ',', 'not', 'gasoline', '.']


In [15]:
print(vars(train.examples[-1])['SRC'])

['une', 'perpignanaise', 'vit', '12', 'ans', 'sans', 'eau', 'courante', ',', 'la', 'saur', 'condamnée', ' ', 'ces', '200', '€', 'ça', 'doit', 'plus', 'ou', 'moins', 'être', 'une', 'consommation', 'annuelle', '(', 'à', 'la', 'louche', '100m3', ')', ',', "c'", 'est', 'de', "l'", 'eau', 'pas', 'de', "l'", 'essence', '.']


In [0]:
def train_attn(model, iterator, optimizer, criterion, clip):
    manual_seed = 77
    torch.manual_seed(manual_seed)
    if n_gpu > 0:
        torch.cuda.manual_seed(manual_seed)
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.SRC
        trg = batch.TRG
        
        optimizer.zero_grad()
        
        output,_ = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate_attn(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.SRC
            trg = batch.TRG

            output,_ = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def init_weights(m):
    manual_seed = 77
    torch.manual_seed(manual_seed)
    if n_gpu > 0:
        torch.cuda.manual_seed(manual_seed)
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def inference(model, file_name, src_vocab, trg_vocab, attention= False, max_trg_len = 64):
    '''
    Function for translation inference

    Input: 
    model: translation model;
    file_name: the directoy of test file that the first column is target reference, and the second column is source language;
    trg_vocab: Target torchtext Field
    attention: the model returns attention weights or not, default = False
    max_trg_len: the maximal length of translation text (optinal), default = 64

    Output:
    Corpus BLEU score.
    '''
    from nltk.translate.bleu_score import corpus_bleu
    from nltk.translate.bleu_score import sentence_bleu
    from torchtext.data import TabularDataset
    from torchtext.data import Iterator

    # convert index to text string
    def convert_itos(convert_vocab, token_ids):
        list_string = []
        for i in token_ids:
            if i == convert_vocab.vocab.stoi['<eos>']:
                break
            else:
                token = convert_vocab.vocab.itos[i]
                list_string.append(token)
        return list_string

        
    test = TabularDataset(
      path=file_name, # the root directory where the data lies
      format='tsv',
      skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
      fields=[('TRG', trg_vocab), ('SRC', src_vocab)])

    test_iter = Iterator(
    dataset = test, # we pass in the datasets we want the iterator to draw data from
    sort = False,batch_size=128,
    sort_key=None,
    shuffle=False,
    sort_within_batch=False,
    device = device,
    train=False
    )
  
    model.eval()
    all_trg = []
    all_translated_trg = []

    TRG_PAD_IDX = trg_vocab.vocab.stoi[trg_vocab.pad_token]

    with torch.no_grad():
    
        for i, batch in enumerate(test_iter):

            src = batch.SRC
            #src = [src len, batch size]

            trg = batch.TRG
            #trg = [trg len, batch size]

            batch_size = trg.shape[1]

            # create a placeholder for traget language with shape of [max_trg_len, batch_size] where all the elements are the index of <pad>. Then send to device
            trg_placeholder = torch.Tensor(max_trg_len, batch_size)
            trg_placeholder.fill_(TRG_PAD_IDX)
            trg_placeholder = trg_placeholder.long().to(device)
            if attention == True:
                output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
            else:
                output = model(src, trg_placeholder, 0) #turn off teacher forcing
            # get translation results, we ignor first token <sos> in both translation and target sentences. 
            # output_translate = [(trg len - 1), batch, output dim] output dim is size of target vocabulary.
            output_translate = output[1:]
            # store gold target sentences to a list 
            all_trg.append(trg[1:].cpu())

            # Choose top 1 word from decoder's output, we get the probability and index of the word
            prob, token_id = output_translate.data.topk(1)
            translation_token_id = token_id.squeeze(2).cpu()

            # store gold target sentences to a list 
            all_translated_trg.append(translation_token_id)
      
    all_gold_text = []
    all_translated_text = []
    for i in range(len(all_trg)): 
        cur_gold = all_trg[i]
        cur_translation = all_translated_trg[i]
        for j in range(cur_gold.shape[1]):
            gold_convered_strings = convert_itos(trg_vocab,cur_gold[:,j])
            trans_convered_strings = convert_itos(trg_vocab,cur_translation[:,j])

            all_gold_text.append(gold_convered_strings)
            all_translated_text.append(trans_convered_strings)

    corpus_all_gold_text = [[item] for item in all_gold_text]
    corpus_bleu_score = corpus_bleu(corpus_all_gold_text, all_translated_text)  
    return corpus_bleu_score

In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, n_layers, dropout=dropout, bidirectional = True)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
  
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.lstm(embedded)
        
        return outputs, (hidden, cell)


class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        #self.W_a = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)
        #self.v_a = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):

        hidden = hidden.permute(1, 0, 2)
        
        encoder_outputs = encoder_outputs.permute(1, 2, 0)
        attention_vector = torch.bmm(hidden, encoder_outputs).squeeze(1)
        
        return F.softmax(attention_vector, dim=1)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, dec_hid_dim, n_layers, dropout, attn):
        super().__init__()

        self.emb_dim = emb_dim
        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.lstm = nn.LSTM(emb_dim, dec_hid_dim, n_layers, dropout=dropout, bidirectional = False)
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

        self.f_mid = nn.Linear(dec_hid_dim * 2, dec_hid_dim)
        self.f_output = nn.Linear(dec_hid_dim, output_dim)

        self.attn = attn
        
    def forward(self, input, hidden, cell, encoder_outputs):
             
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        attention_weights = self.attn(hidden, encoder_outputs)
        attention_weights = attention_weights.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(attention_weights, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        concat = torch.cat((hidden, weighted), dim = 2)
        concat = concat.squeeze(0)
        s_prime = self.f_mid(concat)
        prediction = self.f_output(s_prime)
        
        return prediction, hidden, cell, attention_weights

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        all_attention_weights = torch.zeros(trg.shape[1], trg.shape[0]-1, src.shape[0])

        encoder_outputs, (enc_hidden, enc_cell) = self.encoder(src)

        input = trg[0,:]

        #print('Enc_hidden', enc_hidden.shape)

        hidden = torch.cat((enc_hidden[0], enc_hidden[1]), dim = 1).unsqueeze(0)
        cell = torch.cat((enc_cell[0], enc_cell[1]), dim = 1).unsqueeze(0)
        
        for t in range(1, trg_len):
            
            output, hidden, cell, attention_weights = self.decoder(input, hidden, cell, encoder_outputs)
            
            all_attention_weights[:,t-1,:] = attention_weights.squeeze(1)

            outputs[t] = output

            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1) 

            input = trg[t] if teacher_force else top1

        return outputs,all_attention_weights

In [19]:
INPUT_DIM = 6004
OUTPUT_DIM = 6004
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
N_LAYERS = 1
BI_DIRECTION = True
ENC_HID_DIM = 512
DEC_HID_DIM = 512 * 2 # maybe times 2?
ENC_DROPOUT = 0.3
DEC_DROPOUT = 0.3
TEACH_FORCING_RATE = 0.5
LEARNING_RT = 0.001
MAX_EPOCH = 15

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters(),lr=LEARNING_RT)

def init_weights(m):
    manual_seed = 77
    torch.manual_seed(manual_seed)
    if n_gpu > 0:
        torch.cuda.manual_seed(manual_seed)
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ',TRG_PAD_IDX)
## we will ignore the pad token in true target set
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<pad> token index:  1


In [0]:
# CLIP = 1

# best_valid_loss = float('inf')

# for epoch in range(MAX_EPOCH):
    
#     start_time = time.time()
    
#     train_loss = train_attn(model, train_iter, optimizer, criterion, CLIP)
#     valid_loss = evaluate_attn(model, val_iter, criterion)
    
#     end_time = time.time()
    
#     epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
#     # Create checkpoint at end of each epoch
#     state_dict_model = model.state_dict() 
#     state = {
#         'epoch': epoch,
#         'state_dict': state_dict_model,
#         'optimizer': optimizer.state_dict()
#         }

#     # torch.save(state, "./drive/My Drive/Colab Notebooks/ckpt_lab3_attn/seq2seq_"+str(epoch+1)+".pt")

#     print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
#     print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
#     print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

In [0]:
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(MAX_EPOCH):
    
    start_time = time.time()
    
    train_loss = train_attn(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate_attn(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    torch.save(state, "/content/drive/My Drive/Colab Notebooks/Trends Project/epoch_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 22m 22s
	Train Loss: 4.985 | Train PPL: 146.260
	 Val. Loss: 5.225 |  Val. PPL: 185.822
Epoch: 02 | Time: 22m 26s
	Train Loss: 3.815 | Train PPL:  45.399
	 Val. Loss: 4.934 |  Val. PPL: 138.929
Epoch: 03 | Time: 22m 21s
	Train Loss: 3.253 | Train PPL:  25.858
	 Val. Loss: 4.894 |  Val. PPL: 133.474
Epoch: 04 | Time: 22m 17s
	Train Loss: 2.930 | Train PPL:  18.719
	 Val. Loss: 4.727 |  Val. PPL: 112.974
Epoch: 05 | Time: 22m 37s
	Train Loss: 2.686 | Train PPL:  14.669
	 Val. Loss: 4.746 |  Val. PPL: 115.090
Epoch: 06 | Time: 22m 18s
	Train Loss: 2.495 | Train PPL:  12.116
	 Val. Loss: 4.783 |  Val. PPL: 119.496
Epoch: 07 | Time: 22m 7s
	Train Loss: 2.325 | Train PPL:  10.225
	 Val. Loss: 4.924 |  Val. PPL: 137.501
Epoch: 08 | Time: 22m 23s
	Train Loss: 2.184 | Train PPL:   8.878
	 Val. Loss: 4.911 |  Val. PPL: 135.802
Epoch: 09 | Time: 22m 40s
	Train Loss: 2.050 | Train PPL:   7.769
	 Val. Loss: 5.059 |  Val. PPL: 157.468
Epoch: 10 | Time: 22m 39s
	Train Loss: 1.947 | 

In [20]:
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model_best = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters(),lr=LEARNING_RT)

model_best.apply(init_weights)

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ',TRG_PAD_IDX)
## we will ignore the pad token in true target set
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

model_best.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Trends Project/epoch_4.pt")['state_dict'])

print(inference(model_best, "/content/drive/My Drive/Colab Notebooks/Trends Project/data/output_test.tsv", SRC, TRG, True, 64))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<pad> token index:  1
0.19694590142119364


In [23]:
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT, attn)

model_best = Seq2Seq(enc, dec, device).to(device)

optimizer = optim.Adam(model.parameters(),lr=LEARNING_RT)

model_best.apply(init_weights)

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ',TRG_PAD_IDX)
## we will ignore the pad token in true target set
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

model_best.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/Trends Project/epoch_15.pt")['state_dict'])

print(inference(model_best, "/content/drive/My Drive/Colab Notebooks/Trends Project/data/output_test.tsv", SRC, TRG, True, 64))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<pad> token index:  1
0.2026407182435025


In [21]:
path = "/content/drive/My Drive/Colab Notebooks/Trends Project/data/output.tsv"

tests = []
with open(path) as f:
  f = f.readlines()
  for i in range(3):
    print(f[765 + i].split("\t")[1])
    tests.append(f[765 + i].split("\t")[1])

A partir de ce moment la, j'arrive plus a contenir mes sentiments mais je sais que la distance fait que rien ne se passera et j'essaye de passer outre, mais j'y arrive pas.

A partir de la, c'est la descente aux enfers.

A partir de là il est facile de choisir les arguments les plus idiots et de les contredire.



In [22]:
# sents = [sentence_1, sentence_2, sentence_3]

ind = 1
for sent in tests:

  model_best.eval()
  src_token = SRC.preprocess(sent)
  print("src_token:", src_token)
  src_tensor = SRC.process([src_token],device=device)
  print("shape of source language: ", src_tensor.shape)

  trg_token = ['<pad>']*64
  trg_tensor = TRG.process([trg_token],device=device)

  output, all_attention_weights = model(src_tensor, trg_tensor,teacher_forcing_ratio = 0.0)
  output_dim = output.shape[-1] 
  output_translate = output[1:]

  source_language = src_tensor[:,0].cpu().numpy()
  translation_logit = output_translate[:,0,:].squeeze(1)
  weights = all_attention_weights[0,:,:].squeeze(1).cpu().detach().numpy()

  prob, token_id = translation_logit.data.topk(1)
  token_id = token_id.squeeze(1).cpu().numpy()

  src_language = []

  for i in source_language:
      if i == SRC.vocab.stoi['<eos>']:
          break
      else:
          token = SRC.vocab.itos[i]
          src_language.append(token)
  print("Source language:", src_language)

  trans_language = []

  for i in token_id:
      if i == TRG.vocab.stoi['<eos>']:
          break
      else:
          token = TRG.vocab.itos[i]
          trans_language.append(token)
  print("\n\tOur model translation: ", trans_language)

src_token: ['a', 'partir', 'de', 'ce', 'moment', 'la', ',', "j'", 'arrive', 'plus', 'a', 'contenir', 'mes', 'sentiments', 'mais', 'je', 'sais', 'que', 'la', 'distance', 'fait', 'que', 'rien', 'ne', 'se', 'passera', 'et', "j'", 'essaye', 'de', 'passer', 'outre', ',', 'mais', "j'", 'y', 'arrive', 'pas', '.']
shape of source language:  torch.Size([41, 1])
Source language: ['<sos>', 'a', 'partir', 'de', 'ce', 'moment', 'la', ',', "j'", 'arrive', 'plus', 'a', 'contenir', 'mes', 'sentiments', 'mais', 'je', 'sais', 'que', 'la', 'distance', 'fait', 'que', 'rien', 'ne', 'se', 'passera', 'et', "j'", 'essaye', 'de', 'passer', 'outre', ',', 'mais', "j'", 'y', 'arrive', 'pas', '.']

	Our model translation:  ['une', 'presse', 'create', 'create', 'documentary', '3', 'final', 'camembert', 'hairs', 'openly', 'heating', 'avoided', 'earlier', 'bears', 'create', 'internship', 'create', 'meeting', 'none', 'oppose', 'shout', 'gen2', 'telus', 'pretend', 'pretend', 'concepts', 'cg', 'light', 'distinguish', 'p